In [ ]:
import gymnasium as gym
import ale_py
from stable_baselines3 import DQN


gym.register_envs(ale_py)  # 注册 ALE 环境
env = gym.make("ALE/SpaceInvaders-v5", render_mode=None)

# 采用CNN策略
model = DQN("CnnPolicy", env, verbose=1, buffer_size=100000)
model.learn(total_timesteps=1e7, log_interval=10)
model.save("ale_space_invaders")

2025-05-09 20:04:01.058055: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-09 20:04:01.099164: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-09 20:04:01.780263: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
A.L.E: Arcade Learning Environment (version 0.11.0+dfae0bd)
[Powered by Stella]


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 472      |
|    ep_rew_mean      | 130      |
|    exploration_rate | 0.996    |
| time/               |          |
|    episodes         | 10       |
|    fps              | 527      |
|    time_elapsed     | 8        |
|    total_timesteps  | 4718     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000462 |
|    n_updates        | 1154     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 447      |
|    ep_rew_mean      | 108      |
|    exploration_rate | 0.992    |
| time/               |          |
|    episodes         | 20       |
|    fps              | 547      |
|    time_elapsed     | 16       |
|    total_timesteps  | 

In [1]:
import gymnasium as gym
import ale_py
from stable_baselines3 import DQN

gym.register_envs(ale_py)  # 注册 ALE 环境
env = gym.make("ALE/SpaceInvaders-v5", render_mode="human")
model = DQN.load("ale_space_invaders")

obs, info = env.reset()
total_reward = 0
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(int(action))
    total_reward += reward
    if terminated or truncated:
        obs, info = env.reset()
        print(f"Episode done, reward: {total_reward}")
        print(f"Episode done, truncated: {truncated}, terminated: {terminated}")
        break  # exit(0)


2025-05-09 20:42:55.844925: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-09 20:42:56.549314: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-09 20:42:57.697167: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
A.L.E: Arcade Learning Environment (version 0.11.0+dfae0bd)
[Powered by Stella]
/usr/local/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` 

Episode done, reward: 365.0
Episode done, truncated: False, terminated: True


In [2]:
import gymnasium as gym
from stable_baselines3 import DQN

# 创建测试环境（启用渲染以观察行为）
test_env = gym.make(
    "ALE/SpaceInvaders-v5",
    render_mode="human",  # 或 "rgb_array" 用于无界面渲染
    max_episode_steps=1000  # 与训练参数一致
)

# 加载新训练的模型
model = DQN.load("ale_space_invaders", env=test_env)

# 运行测试并计算平均奖励
total_rewards = []
num_episodes = 10  # 测试回合数

for _ in range(num_episodes):
    obs, _ = test_env.reset()
    done = False
    episode_reward = 0
    while not done:
        action, _ = model.predict(obs, deterministic=True)  # 关闭探索
        obs, reward, done, _, _ = test_env.step(action)
        episode_reward += reward
    total_rewards.append(episode_reward)
    test_env.close()  # 关闭当前回合的环境

# 输出结果
average_reward = sum(total_rewards) / num_episodes
print(f"平均奖励（{num_episodes}回合）: {average_reward:.1f}")

/usr/local/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code expected at most 16 arguments, got 18
  warnings.warn(
/usr/local/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object exploration_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code expected at most 16 arguments, got 18
  warnings.warn(


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
平均奖励（10回合）: 345.5


In [4]:
import torch
import gymnasium as gym
from stable_baselines3 import DQN

# 注册ALE环境并创建训练环境（与原始训练一致）
env = gym.make("ALE/SpaceInvaders-v5", render_mode=None)

# 初始化DQN模型（保持与原训练相同的参数）
model = DQN(
    "CnnPolicy",
    env,
    verbose=1,
    buffer_size=100000,  # 保持与原训练一致
    learning_rate=1e-4,  # 显式指定参数（若原训练未设置则需保持一致）
    exploration_final_eps=0.01,  # 确保探索率参数明确
    device="cuda" if torch.cuda.is_available() else "cpu"  # 启用GPU加速
)

# 训练模型（修改为 total_timesteps=1e5）
model.learn(
    total_timesteps=int(1e5),
    log_interval=10,          # 每10步输出一次日志
    progress_bar=True         # 显示进度条（可选）
)

# 保存新训练的模型（文件名区分原模型）
model.save("ale_spaceinvaders_1e5")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


Output()

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 577      |
|    ep_rew_mean      | 152      |
|    exploration_rate | 0.429    |
| time/               |          |
|    episodes         | 10       |
|    fps              | 431      |
|    time_elapsed     | 13       |
|    total_timesteps  | 5770     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.921    |
|    n_updates        | 1417     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 532      |
|    ep_rew_mean      | 157      |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 20       |
|    fps              | 391      |
|    time_elapsed     | 27       |
|    total_timesteps  | 10649    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.294    |
|    n_updates      

In [5]:
import gymnasium as gym
from stable_baselines3 import DQN

# 创建测试环境（启用渲染以观察行为）
test_env = gym.make(
    "ALE/SpaceInvaders-v5",
    render_mode="human",  # 或 "rgb_array" 用于无界面渲染
    max_episode_steps=1000  # 与训练参数一致
)

# 加载新训练的模型
model = DQN.load("ale_spaceinvaders_1e5", env=test_env)

# 运行测试并计算平均奖励
total_rewards = []
num_episodes = 10  # 测试回合数

for _ in range(num_episodes):
    obs, _ = test_env.reset()
    done = False
    episode_reward = 0
    while not done:
        action, _ = model.predict(obs, deterministic=True)  # 关闭探索
        obs, reward, done, _, _ = test_env.step(action)
        episode_reward += reward
    total_rewards.append(episode_reward)
    test_env.close()  # 关闭当前回合的环境

# 输出结果
average_reward = sum(total_rewards) / num_episodes
print(f"平均奖励（{num_episodes}回合）: {average_reward:.1f}")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


/usr/local/lib/python3.10/site-packages/stable_baselines3/common/buffers.py:242: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 20.16GB > 6.50GB
  warnings.warn(


平均奖励（10回合）: 242.0


In [6]:
import gymnasium as gym
import ale_py
from stable_baselines3 import DQN

gym.register_envs(ale_py)  # 注册 ALE 环境
env = gym.make("ALE/SpaceInvaders-v5", render_mode="human")
model = DQN.load("ale_spaceinvaders_1e5")

obs, info = env.reset()
total_reward = 0
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(int(action))
    total_reward += reward
    if terminated or truncated:
        obs, info = env.reset()
        print(f"Episode done, reward: {total_reward}")
        print(f"Episode done, truncated: {truncated}, terminated: {terminated}")
        break  # exit(0)

/usr/local/lib/python3.10/site-packages/stable_baselines3/common/buffers.py:242: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 20.16GB > 6.49GB
  warnings.warn(


Episode done, reward: 140.0
Episode done, truncated: False, terminated: True
